# Интеллектуальный анализ данных – весна 2025
# Домашнее задание 6: классификация текстов

Правила:



*   Домашнее задание оценивается в 10 баллов.
*   Можно использовать без доказательства любые результаты, встречавшиеся на лекциях или семинарах по курсу, если получение этих результатов не является вопросом задания.
*  Можно использовать любые свободные источники с *обязательным* указанием ссылки на них.
*  Плагиат не допускается. При обнаружении случаев списывания, 0 за работу выставляется всем участникам нарушения, даже если можно установить, кто у кого списал.
*  Старайтесь сделать код как можно более оптимальным. В частности, будет штрафоваться использование циклов в тех случаях, когда операцию можно совершить при помощи инструментов библиотек, о которых рассказывалось в курсе.
* Если в задании есть вопрос на рассуждение, то за отсутствие ответа на него балл за задание будет снижен вполовину.

В этом домашнем задании вам предстоит построить классификатор текстов.

Будем предсказывать эмоциональную окраску твиттов о коронавирусе.



In [2]:
import numpy as np
import pandas as pd
from typing import  List
import matplotlib.pyplot as plt
import seaborn as sns
from string import punctuation

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/hse-ds/iad-intro-ds/refs/heads/master/2025/homeworks/hw06_texts/tweets_coronavirus.csv', encoding='latin-1')
df.sample(4)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
7020,12335,57287,"London, England",19-03-2020,@sunny_hundal @sainsburys is my regular superm...,Negative
30143,40828,85780,"New York, NY",10-04-2020,This piece from @reuters talks about inflation...,Extremely Negative
24882,34254,79206,"Norwich, England",06-04-2020,UK why aren t you matching physical copy price...,Negative
11332,17544,62496,Britain,21-03-2020,#CoronaCrisis stop using trolleys in the super...,Negative


Для каждого твитта указано:


*   UserName - имя пользователя, заменено на целое число для анонимности
*   ScreenName - отображающееся имя пользователя, заменено на целое число для анонимности
*   Location - местоположение
*   TweetAt - дата создания твитта
*   OriginalTweet - текст твитта
*   Sentiment - эмоциональная окраска твитта (целевая переменная)



## Задание 1 Подготовка (0.5 балла)

Целевая переменная находится в колонке `Sentiment`.  Преобразуйте ее таким образом, чтобы она стала бинарной: 1 - если у твитта положительная или очень положительная эмоциональная окраска и 0 - если отрицательная или очень отрицательная.

In [4]:
df['Sentiment'] = df['Sentiment'].replace({'Positive': 1, 'Extremely Positive': 1, 'Extremely Negative': 0, 'Negative': 0})
df.sample(10)

/var/folders/wk/n09m_gw178zflz6sw7crwhz40000gn/T/ipykernel_3198/1831355944.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sentiment'] = df['Sentiment'].replace({'Positive': 1, 'Extremely Positive': 1, 'Extremely Negative': 0, 'Negative': 0})


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
6705,11947,56899,Singapore,19-03-2020,Luxury seafood supply hit by axed flights as #...,0
5887,10946,55898,"Chicago, IL",19-03-2020,Grocery store workers deserve hazard pay!!! #c...,0
16780,24230,69182,NaN,25-03-2020,We would like to dedicate a post to thank to o...,1
21254,29722,74674,NaN,01-04-2020,COVID 19 Ambulance Workers Association on Str...,1
12291,18702,63654,Melbourne & Regional Victoria,22-03-2020,Australians seem oblivious to the risks of 19 ...,0
3342,7851,52803,United Kingdom,18-03-2020,I only booked my hotel for only 2 days to see ...,1
25373,34872,79824,Once Great Southern Land,06-04-2020,Wouldn't buy any from this maggot OR the super...,0
32018,43186,88138,NaN,12-04-2020,"Dropping fuel prices during COVID-19, but shou...",0
25385,34891,79843,Iraq,06-04-2020,2) OPEC+ et al cannot stop the damage generate...,1
10995,17139,62091,NaN,21-03-2020,No grooming. No eating out. No online shopping...,0


Сбалансированы ли классы?

In [5]:
df['Sentiment'].value_counts()

Sentiment
1    18046
0    15398
Name: count, dtype: int64

Ну вообще, разница в 12%≈ не так критична ([источник гугл](https://developers.google.com/machine-learning/crash-course/overfitting/imbalanced-datasets?hl=ru)). Поэтому их можно назвать сбалансированными





:Выведете на экран информацию о пропусках в данных. Если пропуски присутствуют заполните их строкой 'Unknown'.

In [6]:
df.isnull()

df[df.isnull()] = 'Unknown'
df.sample(3)


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
14836,21840,66792,France,23-03-2020,Marketers designing messaging to ease and enga...,1
28780,39116,84068,Forodwaith,09-04-2020,If you suspect a Scam please report it to the ...,0
16477,23861,68813,Unknown,24-03-2020,@acgrayling Despite Covid-19 pandemic deaths a...,0


In [7]:
df.isnull().value_counts()

UserName  ScreenName  Location  TweetAt  OriginalTweet  Sentiment
False     False       False     False    False          False        33444
Name: count, dtype: int64

Разделите данные на обучающие и тестовые в соотношении 7 : 3 и

укажите `random_state=0`

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(
    df,
    test_size = 0.3,
    random_state = 0
)

## Задание 2 Токенизация (3 балла)

Постройте словарь на основе обучающей выборки и посчитайте количество встреч каждого токена с использованием самой простой токенизации - деления текстов по пробельным символам и приведения токенов в нижний регистр.

In [9]:
bag=[]
for i in train['OriginalTweet']:
    lowi = i.lower()
    tokens = lowi.split()
    for token in tokens:
        bag.append(token)

bag[0:30]

['why',
 'we',
 'still',
 'want',
 'to',
 'buy',
 'so',
 'much',
 'stuff',
 'during',
 'quarantine',
 'https://t.co/1m881cwfuv',
 '#shopping',
 '#covid_19',
 '#online',
 'with',
 'driving',
 'even',
 'more',
 'usage',
 'a',
 'strong',
 'strategy',
 'is',
 'critical',
 'now',
 'more',
 'then',
 'ever',
 'luckily']

Какой размер словаря получился?

In [10]:
len(bag)

754630

Выведите 10 самых популярных токенов с количеством встреч каждого из них. Объясните, почему именно эти токены в топе.

In [11]:
from collections import Counter
freq = Counter(bag)
top10 = freq.most_common(10)
top10


[('the', 26815),
 ('to', 23373),
 ('and', 14684),
 ('of', 13012),
 ('a', 11737),
 ('in', 11198),
 ('for', 8566),
 ('#coronavirus', 8223),
 ('is', 7383),
 ('are', 7050)]

**Ответ:** # Мы видим слова, которые наиболее часто используются в речи и предложениях, особенно в английском яызке. Например, артикли the и a встречаются повсеместно по всему языку, хотя и не несут значимой для анализа (чаще всего) пользы.

Из интересного #coronavirus часто встречался в контексте. Даже чаще, чем is и are. Это может говорить о том, что датасет вероятно с ковидных времен, когда об этом говорили постоянно)

Удалите стоп-слова из словаря и выведите новый топ-10 токенов (и количество встреч) по популярности.  Что можно сказать  о нем?

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

pd_bag = pd.Series(bag)
fbag = pd_bag[~pd_bag.isin(stopwords.words())]

fbag.value_counts().head(10)

#coronavirus    8223
prices          3891
food            3820
grocery         3469
supermarket     3288
covid-19        3173
store           3155
#covid19        2471
&amp;           2314
consumer        2245
Name: count, dtype: int64

**Ответ:**  Ну как я и подозревал, все оказалось словами-филлерами в предыдущем примере.  В общем, по топ10 явно видно, что бушевал ковид, люди жаловались на цены, еду, продукты и супермаркеты.

НО! Как-то в примеры просочился амперсанд, это не очень хороший знак(в плане образно, а не фигурально). Либо твиттер его использует при индексации, либо люди любят часто его писать. Формально, наверное, это можно считать stop-word'ом... иииии.. Вероятно, его можно выскрести с помощью хорошего токенайзера с помощью nltk.

Также выведите 20 самых непопулярных слов (если самых непопулярных слов больше, выведите любые 20 из них) Почему эти токены непопулярны, требуется ли как-то дополнительно работать с ними?

In [13]:
fbag.value_counts().tail(20)

50kg                       1
466                        1
pmt                        1
acres                      1
cucumber                   1
https://t.co/x69jqbisox    1
#springcleaning            1
587-4272                   1
massive.                   1
https://t.co/l8jnzxjgwo    1
readiness,                 1
kick-in                    1
(covid-19,                 1
impacts)                   1
https://t.co/wa7kcdwqea    1
@linkedin                  1
clare                      1
connors                    1
levins                     1
https://t.co/7j2y3rsld9    1
Name: count, dtype: int64

**Ответ:** Внимание! На горизонте всякий мусор. Мы видим кучу ссылок, странных штук что слепились... А еще страну в Азии под названием Бурма. Слышали когда-нибудь об этом? Я тоже.. нет.. Но вот огурцы похоже твиттеряне не долюбливают.

По-хорошему, нужно почистить от всяких ссылок неприятных массив данных. А еще ВОСПОЛЬЗОВАТЬСЯ НОРМАЛЬНЫМ ТОКЕНАЙЗЕРОМ, чтобы не было таких (кейсов?"!)





Теперь воспользуемся токенайзером получше - TweetTokenizer из библиотеки nltk. Примените его и посмотрите на топ-10 популярных слов. Чем он отличается от топа, который получался раньше? Почему?

In [14]:
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

tokens = fbag.apply(tokenizer.tokenize).explode()
# nt = tokens[~tokens.isin(stopwords.words())]
tokens.value_counts().head(10)

.               24418
,               17571
?                9524
#coronavirus     8808
â                7415
                7311
19               7167
-                6643
covid            6253
prices           4601
Name: count, dtype: int64

**Ответ:** Надо поубирать стопслова и пунктуацию. У нас появились всякие точки, запятые, и прочее.. Потому что токенайзером воспользовались, а не просто поделили по пробелам

Из плюсов, видно как ушли всякие **подобные?%;№"! !.штуковины и амперсанды!!!!**

но ссылки остались:(((

Для меня загадка – число 19 (А СТОП КОВИД-19 ЖЕ, забыли),
а еще "â" (видимо какой-то испанский прикол)



In [15]:
tokens.value_counts().tail(20)

@gordongchang              1
logarithmic                1
#foodretail                1
non-log                    1
improveã                   1
https://t.co/aacxqg5sej    1
rebooked                   1
can't-miss                 1
https://t.co/8xikga3rel    1
@awgcorporate              1
https://t.co/fglsnpoj4k    1
https://t.co/dj8jvtzrs7    1
https://t.co/cuu8ff09pg    1
https://t.co/gyogjmwo68    1
shoukd                     1
https://t.co/vpgmpxrgba    1
distsnce                   1
resteraunts                1
crams                      1
https://t.co/7j2y3rsld9    1
Name: count, dtype: int64

Удалите из словаря стоп-слова и пунктуацию, посмотрите на новый топ-10 слов с количеством встреч, есть ли теперь в нем что-то не похожее на слова?

In [16]:
from string import punctuation

# punctuation?

nt = tokens[~tokens.isin(list(punctuation))]
nt = nt[~nt.isin(stopwords.words())]

nt.value_counts().head(10)

#coronavirus    8808
â               7415
               7311
19              7167
covid           6253
prices          4601
               4372
food            4367
store           3877
supermarket     3805
Name: count, dtype: int64

**Ответ:** Да, мы видим "" или  "  " и все еще загадочный "â". Согласен, достаточно немногословно. Вероятно какие-то символы для оформления, либо особенности оформления.

Скорее всего в некоторых топах были неотображаемые символы или отдельные буквы не латинского алфавита. Уберем их: удалите из словаря токены из одного символа, позиция которого в таблице Unicode 128 и более (`ord(x) >= 128`)

Выведите топ-10 самых популярных и топ-20 непопулярных слов. Чем полученные топы отличаются от итоговых топов, полученных при использовании токенизации по пробелам? Что теперь лучше, а что хуже?

In [17]:
nt = nt[~nt.apply(lambda x: len(x) == 1 and ord(x) >= 128)]
nt.value_counts().head(10)

#coronavirus    8808
19              7167
covid           6253
prices          4601
food            4367
store           3877
supermarket     3805
grocery         3523
#covid19        2589
consumer        2340
Name: count, dtype: int64

In [18]:
nt.value_counts().tail(20)

#nevada                    1
@frickindistant            1
synopsis                   1
equation                   1
hendrickson                1
https://t.co/hq23xalxjo    1
https://t.co/vy6hr3ui76    1
biso                       1
@robertgordonuni           1
https://t.co/tfnetiq3uq    1
https://t.co/njh8dbwdxc    1
power-up                   1
callaway                   1
@krcg13                    1
https://t.co/shevekhipn    1
michiganders               1
whitless                   1
@aifytheresa               1
https://t.co/etvxjkjooa    1
https://t.co/7j2y3rsld9    1
Name: count, dtype: int64

**Ответ:** Ну слушайте, это уже очень хорошо, мы избавились от всякого мусора и остались удобоваримые приятные вещи. Все еще сильно засоряют хештеги пространство. Отличаются от пробельных тем, что нет всякого мусора ни в самих токенах, ни отдельно. Это приятно, с этим можно работать.



Выведите топ-10 популярных хештегов (токены, первые символы которых - #) с количеством встреч. Что можно сказать о них?

In [19]:
nt[nt.apply(lambda x: x[0] == '#')].value_counts().head(10)

#coronavirus            8808
#covid19                2589
#covid_19               1734
#covid2019               946
#toiletpaper             744
#covid                   641
#socialdistancing        465
#coronacrisis            448
#pandemic                257
#coronaviruspandemic     249
Name: count, dtype: int64

**Ответ:** #toiletpaper единственный хештег, косвенный к упоминанию ковида. многое говорит о нашем обществе...............

Мы видим, что в те времена царил ковид и самыми главными темами было обсуждение и упомянание ковида.

То же самое проделайте для ссылок на сайт https://t.co Сравнима ли популярность ссылок с популярностью хештегов? Будет ли информация о ссылке на конкретную страницу полезна?

In [20]:
nt[nt.apply(lambda x: x.startswith('https://t.co'))].value_counts().head(10)

https://t.co/oxa7swtond    5
https://t.co/gp3eusapl8    4
https://t.co/kuwipf1kqw    3
https://t.co/bylqxrjmnt    3
https://t.co/wuieefsnoj    3
https://t.co/zjnrx6dkkn    3
https://t.co/wrlhyzizaa    3
https://t.co/g63rp042ho    3
https://t.co/e2znxajpre    3
https://t.co/catkegayoy    3
Name: count, dtype: int64

**Ответ:** # Очевидно, популярность у таких штук будет ощутимо меньше. Это логично, ведь все ссылки уникальны. Редко, когда будут делиться чем-то одним и тем же.
Для обучения они нам не нужны и бесполезны. Убираем

Используем опыт предыдущих экспериментов и напишем собственный токенайзер, улучшив TweetTokenizer. Функция tokenize должна:



*   Привести текст в нижний регистр
*   Применить TweetTokenizer для  выделения токенов
*   Удалить стоп-слова, пунктуацию, токены из одного символа с позицией в таблице Unicode 128 и более,  ссылки на t.co



In [21]:
def custom_tokenizer(text):
  tokens = pd.Series(text.lower())
  tokens = tokens.apply(tokenizer.tokenize).explode()

  nt = tokens[~tokens.isin(list(punctuation))]
  nt = nt[~nt.isin(stopwords.words())]

  nt = nt[~nt.apply(lambda x: len(x) == 1 and ord(x) >= 128)]

  nt = nt[~(nt.apply(lambda x: x.startswith('https://t.co')))]

  tokens = nt.to_list()
  return tokens


In [22]:
custom_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext')

['sample', 'text', '@sample_text', '#sampletext']

## Задание 3 Векторизация текстов (2 балла)

Обучите CountVectorizer с использованием custom_tokenizer в качестве токенайзера. Как размер полученного словаря соотносится с размером изначального словаря из начала задания 2?

In [23]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer = custom_tokenizer)
cv.fit(train['OriginalTweet'])

print(len(cv.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


44498


**Ответ:** слов стало гораздо меньше потому что мы убрали с помощью токенайзера весь мусор.

Посмотрим на какой-нибудь конкретный твитт:

In [24]:
ind = 9023
train.iloc[ind]['OriginalTweet'], train.iloc[ind]['Sentiment']

('Nice one @SkyNews lets not panic but show ppl in france queueing for food!!! #CoronavirusOutbreak #COVID2019 brainless!! Ffs',
 np.int64(0))

Автор твитта не доволен ситуацией с едой во Франции и текст имеет резко негативную окраску.

Примените обученный CountVectorizer для векторизации данного текста, и попытайтесь определить самый важный токен и самый неважный токен (токен, компонента которого в векторе максимальна/минимальна, без учета 0). Хорошо ли они определились, почему?

In [27]:
tweet = train.iloc[ind]['OriginalTweet']
vec = cv.transform([tweet]).toarray().flatten()
features = cv.get_feature_names_out()
idx = np.nonzero(vec)[0]

max_idx = idx[vec[idx].argmax()]
min_idx = idx[vec[idx].argmin()]

print(f"{features[max_idx]} – важный (частота {vec[max_idx]})")
print(f"{features[min_idx]} – неважный (частота {vec[min_idx]})")

#coronavirusoutbreak – важный (частота 1)
#coronavirusoutbreak – неважный (частота 1)


**Ответ:** Ну похоже наш countvectorizer бесполезный.. Все по единичке (я проверил)... BoW смотрит же на повторяемосьь.. Значит надо иначе

Теперь примените TfidfVectorizer и  определите самый важный/неважный токены. Хорошо ли определились, почему?

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(tokenizer = custom_tokenizer)
tfidf.fit(train['OriginalTweet'])

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function custom_tokenizer at 0x144b4e340>)

In [29]:
tweet = train.iloc[ind]['OriginalTweet']
vec = tfidf.transform([tweet]).toarray().flatten()
features = tfidf.get_feature_names_out()
idx = np.nonzero(vec)[0]

max_idx = idx[vec[idx].argmax()]
min_idx = idx[vec[idx].argmin()]

print(f"{features[max_idx]} – важный (частота {vec[max_idx]})")
print(f"{features[min_idx]} – неважный (частота {vec[min_idx]})")

brainless – важный (частота 0.39201120274014334)
food – неважный (частота 0.1148258797042418)


**Ответ:** Здесь мы уже видим явное определение. Метод TF IDF работает лучше потому что использует другую стратегию. Он учитывает значимость слов глобально, в то время как countvectorizer просто считает, есть ли слово (1 и 0), вообще не задумываясь о важности и контексте



Найдите какой-нибудь положительно окрашенный твитт, где TfidfVectorizer хорошо (полезно для определения окраски) выделяет важный токен, поясните пример.

*Подсказка:* явно положительные твитты можно искать при помощи положительных слов (good, great, amazing и т. д.)

In [30]:
tweetPositive = train['OriginalTweet'][23284]

In [31]:
train[train['OriginalTweet'].apply(lambda x: 'enjoy' in x) & (train['Sentiment'] == 1)]

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
20526,28843,73795,"ÃÂT: 51.4761159,-2.5612165",29-03-2020,I'm getting pretty fucking sick of celebs tell...,1
21131,29571,74523,"Wales, United Kingdom",01-04-2020,"Check out our recent blog post on Coronavirus,...",1
15590,22757,67709,Riyadh- London-Reading,24-03-2020,"@Harrods Your store is the best to me, and it ...",1
282,4143,49095,Unknown,16-03-2020,This guy is taking the whole #coronavirus in h...,1
20788,29157,74109,"San Rafael, CA",31-03-2020,"""As we practice social distancing, stock food ...",1
18368,26144,71096,Unknown,25-03-2020,@Antcon7062 Seriously ? what is closing off li...,1
18229,25978,70930,"Highland, UT",25-03-2020,"Ya'll ever use so much hand sanitizer, that yo...",1
32073,43255,88207,Brasil,13-04-2020,Good morning on We hope you re enjoying your w...,1
4399,9147,54099,Unknown,18-03-2020,Please avoid unnecessary movement and enjoy th...,1
17351,24909,69861,Unknown,25-03-2020,@katiecouric Stuck at home without Covid-19 te...,1


In [32]:
tweetPositive

'Some Australian farmers are enjoying five-fold spikes in direct-to-consumer produce sales due to #coronavirus: https://t.co/8xFCpFe387 #buylocal'

In [33]:

vecpos = tfidf.transform([tweetPositive]).toarray().flatten()
features = tfidf.get_feature_names_out()
idxpos = np.nonzero(vecpos)[0]

max_idx = idxpos[vecpos[idxpos].argmax()]
min_idx = idxpos[vecpos[idxpos].argmin()]

print(f"{features[max_idx]} – важный (частота {vecpos[max_idx]})")
print(f"{features[min_idx]} – неважный (частота {vecpos[min_idx]})")

five-fold – важный (частота 0.4314939277907398)
#coronavirus – неважный (частота 0.08252535695941979)


**Ответ:**  five-fold встречается в текстах достаточно редко, что выделяет этот твит срди других. #coronavirus, например, налборот есть везде, поэтому имеет низкую частоту TF IDF

## Задание 4 Обучение первых моделей (1 балл)

Примените оба векторайзера для получения матриц с признаками текстов.  Выделите целевую переменную.

In [39]:
X_train = train['OriginalTweet']
y_train = train['Sentiment']

X_test = test['OriginalTweet']
y_test = test['Sentiment']

In [40]:
tfidf.fit(X_train)
cv.fit(X_train)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function custom_tokenizer at 0x144b4e340>)

In [41]:
X_tr_tfidf = tfidf.transform(X_train)
X_te_tfidf = tfidf.transform(X_test)

X_tr_cv = cv.transform(X_train)
X_te_cv = cv.transform(X_test)


Обучите логистическую регрессию на векторах из обоих векторайзеров. Посчитайте долю правильных ответов на обучающих и тестовых данных. Какой векторайзер показал лучший результат? Что можно сказать о моделях?

Используйте `sparse` матрицы (после векторизации), не превращайте их в `numpy.ndarray` или `pd.DataFrame` - может не хватить памяти.

In [50]:
print('tfidf')
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

classif = LogisticRegression(max_iter=10000)
classif.fit(X_tr_tfidf, y_train)

y_pred_train = classif.predict(X_tr_tfidf)
y_pred_test = classif.predict(X_te_tfidf)

print(f'train accurast: {accuracy_score(y_train, y_pred_train)}')
print(f'test accuracy: {accuracy_score(y_test, y_pred_test)}')

tfidf
train accurast: 0.9044852627082444
test accuracy: 0.8212078931632449


In [51]:
print('cv')
classcv = LogisticRegression(max_iter=10000)
classcv.fit(X_tr_cv, y_train)

y_pred_train1 = classcv.predict(X_tr_cv)
y_pred_test1 = classcv.predict(X_te_cv)

print(f'train accurast: {accuracy_score(y_train, y_pred_train1)}')
print(f'test accuracy: {accuracy_score(y_test, y_pred_test1)}')

cv
train accurast: 0.9739854762921828
test accuracy: 0.8335658760215268


**Ответ:** CV лучше, чем tfidf в данном случае. работаем дальше с ним
Также мы видим, что CV очень хорошо распознает train. Как бы ни хорошо, ни плохо
Обе модели спользуют одну и ту же регуляризацию. В теории, если поиграться с рег., то можно получить и другие значнеия но меня уже все устраивает


## Задание 5 Стемминг (0.5 балла)

Для уменьшения словаря можно использовать стемминг.

Модифицируйте написанный токенайзер, добавив в него стемминг с использованием SnowballStemmer. Обучите Count- и Tfidf- векторайзеры. Как изменился размер словаря?

In [52]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

In [67]:
def custom_stem_tokenizer(text):
  tokens_cust = custom_tokenizer(text)
  tokens = []
  for i in tokens_cust:
      tokens.append(stemmer.stem(i))
  return tokens

In [68]:
custom_stem_tokenizer('This is sample text!!!! @Sample_text I, \x92\x92 https://t.co/sample  #sampletext adding more words to check stemming')

['sampl', 'text', '@sample_text', '#sampletext', 'ad', 'word', 'check', 'stem']

In [69]:
cv = CountVectorizer(tokenizer=custom_stem_tokenizer)
cv.fit(train['OriginalTweet'])
cv.transform(train['OriginalTweet'])

tfidf = TfidfVectorizer(tokenizer=custom_stem_tokenizer)
tfidf.fit(train['OriginalTweet'])
tfidf.transform(train['OriginalTweet'])

print(len(cv.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


36104


In [70]:
print(len(tfidf.vocabulary_))

36104


**Ответ** Очевидно, что словарь чут-чут усох. Дело все в том, что снежным комом мы сжали много слов... До 36104 слов в нашем случае

Обучите логистическую регрессию с использованием обоих векторайзеров. Изменилось ли качество? Есть ли смысл применять стемминг?

In [75]:
cv_stem = CountVectorizer(tokenizer=custom_stem_tokenizer)
cv_stem.fit(X_train)

tfidf_stem = TfidfVectorizer(tokenizer=custom_stem_tokenizer)
tfidf_stem.fit(X_train)

X_tr_cv = cv_stem.transform(X_train)
X_te_cv = cv_stem.transform(X_test)
X_tr_tfidf = tfidf_stem.transform(X_train)
X_te_tfidf = tfidf_stem.transform(X_test)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [77]:
print('cv')
classtem_cv = LogisticRegression(max_iter=10000)
classtem_cv.fit(X_tr_cv, y_train)
y_tr_pred_cv = classtem_cv.predict(X_tr_cv)
y_te_pred_cv = classtem_cv.predict(X_te_cv)
print(f'train accurasy: {accuracy_score(y_train, y_tr_pred_cv)}')
print(f'test accuracy: {accuracy_score(y_test, y_te_pred_cv)}')

print('tfidf')
classtf_stem = LogisticRegression(max_iter=10000)
classtf_stem.fit(X_tr_tfidf, y_train)
y_tr_pred_tf = classtf_stem.predict(X_tr_tfidf)
y_te_pred_tf = classtf_stem.predict(X_te_tfidf)
print(f'train accuracy: {accuracy_score(y_train, y_tr_pred_tf)}')
print(f'test accuracy: {accuracy_score(y_test, y_te_pred_tf)}')


cv
train accurasy: 0.9582656984194788
test accuracy: 0.8366553717360973
tfidf
train accuracy: 0.894788551900897
test accuracy: 0.826091289615308


**Ответ:** на тестовой выборке почти не изменилось, а вот на трейн чуть-чуть упало. однако словарь уменьшился, в любом случае, это лучше.




## Задание  6 Работа с частотами (1.5 балла)

Еще один способ уменьшить количество признаков - это использовать параметры min_df и max_df при построении векторайзера  эти параметры помогают ограничить требуемую частоту встречаемости токена в документах.

По умолчанию берутся все токены, которые встретились хотя бы один раз.



Подберите max_df такой, что размер словаря будет ~~36651 (на 1 меньше, чем было)~~ 36103 (потому что там далеко не 36552, а 36104, расхожесть обсуждалась в беседе основного канала). Почему параметр получился такой большой/маленький?

In [88]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=1,
                        max_df=8000
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


36103


In [89]:
tfidf_df = TfidfVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=1,
                        max_df=8000
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(tfidf_df.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


36103


**Ответ:** Ну начнем с того что вообще такое min_df и max_df параметры: min_df = 1 оставляет токен, если он встретился в минимум одном документе. То есть все токены. Этот параметр я стал не сильно ломать, но логика у него такая. max_df же делает так, чтоо токен остается если он в НЕ БОЛЕЕ чем n документов. На 10000 у меня было 36104, а на 8000 36103, соответственно один токен куда-то выпал на этом моменте, потому что встречается больше 8000 раз.

Подберите min_df (используйте дефолтное значение max_df) в CountVectorizer таким образом, чтобы размер словаря был 3700 токенов (при использовании токенайзера со стеммингом), а качество осталось таким же, как и было. Что можно сказать о результатах?

In [98]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=9
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3939


In [99]:
cv_df = CountVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=10
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3680


In [101]:
tfidf_df = TfidfVectorizer(tokenizer=custom_stem_tokenizer,
                        min_df=9
                        ).fit(
                            train['OriginalTweet']
                            )
print(len(cv_df.vocabulary_))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


3680


**Ответ:** 3939 и 3680 с какой стати??? Вероятно из-за разности в количестве слов как и ранее. Так что добиться такого невозможно с текущими данными. Но в любом случае размер мы уменьшили, пояснение за то как работает min_df прикрепил выше.

В предыдущих заданиях признаки не скалировались. Отскалируйте данные (при словаре размера 3.7 тысяч, векторизованные CountVectorizer), обучите логистическую регрессию, посмотрите качество и выведите `barplot`, содержащий по 10 токенов, с наибольшим по модулю положительными/отрицательными весами. Что можно сказать об этих токенах?

## Задание 7 Другие признаки (1.5 балла)

Мы были сконцентрированы на работе с текстами твиттов и не использовали другие признаки - имена пользователя, дату и местоположение

Изучите признаки UserName и ScreenName. полезны ли они? Если полезны, то закодируйте их, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Изучите признак TweetAt в обучающей выборке: преобразуйте его к типу datetime и нарисуйте его гистограмму с разделением по цвету на основе целевой переменной. Полезен ли он? Если полезен, то закодируйте его, добавьте к матрице с отскалированными признаками, обучите логистическую регрессию, замерьте качество.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --



Поработайте с признаком Location в обучающей выборке. Сколько уникальных значений?

In [ ]:
# -- YOUR CODE HERE --

Постройте гистограмму топ-10 по популярности местоположений (исключая Unknown)

In [ ]:
# -- YOUR CODE HERE --

Видно, что многие местоположения включают в себя более точное название места, чем другие (Например, у некоторых стоит London, UK; а у некоторых просто UK или United Kingdom).

Создайте новый признак WiderLocation, который содержит самое широкое местоположение (например, из London, UK должно получиться UK). Сколько уникальных категорий теперь? Постройте аналогичную гистограмму.

In [ ]:
# -- YOUR CODE HERE --

Закодируйте признак WiderLocation с помощью OHE таким образом, чтобы создались только столбцы для местоположений, которые встречаются более одного раза. Сколько таких значений?


In [ ]:
# -- YOUR CODE HERE --

Добавьте этот признак к матрице отскалированных текстовых признаков, обучите логистическую регрессию, замерьте качество. Как оно изменилось? Оказался ли признак полезным?


*Подсказка:* используйте параметр `categories` в энкодере.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --